# DA5401 A7 — Multi-Class Model Selection using ROC & PRC

**Objective:** Create a reproducible analysis that compares multiple classifiers on the UCI Landsat Satellite dataset (6 classes) using multi-class ROC and Precision-Recall curves (One-vs-Rest averaging). The notebook below contains code, explanations, plots, and final recommendation.

**Contents**

1. Data download & preprocessing
2. Baseline training & metrics (Accuracy, Weighted F1)
3. Multi-class ROC (OvR) — macro-averaged plots and AUCs
4. Multi-class Precision-Recall (OvR) — macro-averaged plots and Average Precision
5. Synthesis and recommendation

**Note:** Run the notebook cells in order. The dataset is downloaded from the UCI repository when you run the cells.


In [ ]:
# Dependencies
# Run this cell first. If you run in an environment without xgboost installed, the XGBoost cell will skip gracefully.
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.multiclass import OneVsRestClassifier

# Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Optional
from sklearn.ensemble import RandomForestClassifier
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except Exception as e:
    XGBOOST_AVAILABLE = False

print('xgboost available:', XGBOOST_AVAILABLE)
print('sklearn version:', __import__('sklearn').__version__)

In [ ]:
import pandas as pd

# Define file paths — adjust if they’re in a subfolder
train_path = "sat.trn"
test_path = "sat.tst"

# Column names: 36 features + 1 label
col_names = [f'feat_{i}' for i in range(36)] + ['label']

train = pd.read_csv(train_path, header=None, delim_whitespace=True)
test = pd.read_csv(test_path, header=None, delim_whitespace=True)
df = pd.concat([train, test], axis=0).reset_index(drop=True)
df.columns = col_names
print("Shape:", df.shape)
display(df.head())



In [ ]:
# Preprocessing: drop rows with missing data (if any), filter out 'all types present' if present.
# In the UCI Satimage dataset, labels are integers 1..6. If there's a label representing 'all types present', user should remove it.
df = df.copy()
print('Unique labels before any filtering:', df['label'].unique() if 'label' in df.columns else [])  # helps debug

# If the dataset wasn't downloaded, the dataframe will be empty. Guard against that.
if df.empty:
    raise RuntimeError('Dataframe is empty. Please download the dataset (internet) or provide a local path.')

# Convert label to integer, and if there's a label that denotes 'all types present' (e.g., 7) drop it.
df['label'] = df['label'].astype(int)
# Inspect counts
print('Label counts:')
display(df['label'].value_counts())

# If dataset contains a label not in 1..6, drop it
valid_labels = [1,2,3,4,5,6]
df = df[df['label'].isin(valid_labels)].reset_index(drop=True)
X = df.drop('label', axis=1).values
y = df['label'].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42, stratify=y)
print('Train/test shapes:', X_train.shape, X_test.shape)

In [ ]:
# Train the six required models
models = {
    'KNN': KNeighborsClassifier(),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'Dummy(Prior)': DummyClassifier(strategy='prior', random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, multi_class='ovr', solver='lbfgs'),
    'GaussianNB': GaussianNB(),
    'SVC': SVC(probability=True, random_state=42)
}

trained = {}
for name, clf in models.items():
    clf.fit(X_train, y_train)
    trained[name] = clf
    print(f'Trained {name}')

In [ ]:
# Baseline evaluation: Overall Accuracy and Weighted F1
from pprint import pprint
baseline_metrics = {}
for name, clf in trained.items():
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1w = f1_score(y_test, y_pred, average='weighted')
    baseline_metrics[name] = {'accuracy': acc, 'f1_weighted': f1w}
pprint(baseline_metrics)

In [ ]:
# Multi-class ROC (One-vs-Rest) and macro-averaged ROC plot across models
# Binarize labels for OvR
classes = np.unique(y_train)
y_test_binarized = label_binarize(y_test, classes=classes)

plt.figure(figsize=(10, 8))
for name, clf in trained.items():
    # For OvR ROC we need probability estimates for each class
    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)
    else:
        # For some classifiers that provide decision_function instead:
        if hasattr(clf, "decision_function"):
            try:
                y_score = clf.decision_function(X_test)
                # If shape is (n_samples,), expand
                if y_score.ndim == 1:
                    y_score = np.vstack([1 - y_score, y_score]).T
            except:
                continue
        else:
            continue

    # Compute per-class ROC and AUC
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i, cls in enumerate(classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute macro-average AUC
    # First aggregate all FPR
    all_fpr = np.unique(np.concatenate([fpr[i] for i in fpr]))
    # Then interpolate all ROC at these points
    mean_tpr = np.zeros_like(all_fpr)
    for i in fpr:
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= len(classes)
    macro_auc = auc(all_fpr, mean_tpr)
    plt.plot(all_fpr, mean_tpr, label=f'{name} (macro AUC = {macro_auc:.3f})')

plt.plot([0,1],[0,1],'--', linewidth=1, label='Chance')  # diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('OvR Macro-averaged ROC curves for models')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

**ROC Interpretation**

- The plot above shows macro-averaged ROC curves (One-vs-Rest interpolation) for each model. Each model's macro AUC is printed in the legend.
- Identify the model with the highest macro-averaged AUC from the legend above.
- If any model has AUC < 0.5, that implies it's performing worse than random for the averaged OvR decision; possible reasons include the classifier's predictions being inverted for some classes, class imbalance, or a poorly suited model (e.g., Dummy prior for minority classes).

In [ ]:
# Multi-class Precision-Recall (OvR) and macro-averaged PR plot across models
plt.figure(figsize=(10, 8))
for name, clf in trained.items():
    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)
    else:
        if hasattr(clf, "decision_function"):
            try:
                y_score = clf.decision_function(X_test)
                if y_score.ndim == 1:
                    y_score = np.vstack([1 - y_score, y_score]).T
            except:
                continue
        else:
            continue

    # Compute per-class PR and average precision
    precision = dict()
    recall = dict()
    ap = dict()
    for i, cls in enumerate(classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test_binarized[:, i], y_score[:, i])
        ap[i] = average_precision_score(y_test_binarized[:, i], y_score[:, i])

    # Macro-average precision-recall curve via interpolation on recall
    all_recall = np.unique(np.concatenate([recall[i] for i in recall]))
    mean_precision = np.zeros_like(all_recall)
    for i in recall:
        mean_precision += np.interp(all_recall, recall[i][::-1], precision[i][::-1])  # reverse for monotonicity
    mean_precision /= len(classes)
    macro_ap = np.mean([ap[i] for i in ap])
    plt.step(all_recall, mean_precision, where='post', label=f'{name} (mAP = {macro_ap:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('OvR Macro-averaged Precision-Recall curves for models')
plt.legend(loc='lower left')
plt.grid(True)
plt.ylim([0.0, 1.05])
plt.show()

**PRC Interpretation**

- The legend shows the macro-averaged Average Precision (mAP) for each model.
- The model with the highest mAP is the best in terms of precision-recall balance across classes.
- Poor models tend to have precision dropping quickly as recall increases because they produce low-quality ranked scores: to increase recall they must accept many false positives, which reduces precision.


## Synthesis & Recommendation

Compare rankings from Weighted F1, Macro ROC-AUC, and Macro AP (PRC). Discuss any disagreements (e.g., models with good ROC but low AP) and recommend the best model considering threshold behavior (if you need high precision vs high recall).

**Recommendation template:**
- If the task values balanced performance across classes and threshold-stability, choose the model with consistently high macro-AUC and macro-AP (e.g., LogisticRegression or SVC depending on results).
- If precision for minority classes is critical, prioritize the model with the highest mAP.

---

### Optional: RandomForest and XGBoost (Brownie points)

Add and train RandomForest and XGBoost (if available) and compute the same ROC/PR analyses for comparison.


In [ ]:
# Brownie points: RandomForest and XGBoost (optional)
extra_models = {}
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
extra_models['RandomForest'] = rf
print('Trained RandomForest')

if XGBOOST_AVAILABLE:
    xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    xgb_clf.fit(X_train, y_train)
    extra_models['XGBoost'] = xgb_clf
    print('Trained XGBoost')
else:
    print('XGBoost not available in this environment. To use it, install xgboost and re-run this cell.')

# You can append extra_models to trained and re-run the ROC/PR plotting cells if desired.

### Save & Download

Run **File -> Save notebook** in your Jupyter environment. The notebook file can be downloaded from the environment interface or from the path `/mnt/data/DA5401_A7_MultiClass_ROC_PRC.ipynb` if you run this notebook in a standard Jupyter environment.

---

Good luck! If you want, I can run this notebook here (execute all cells) and attach the generated output, but note that internet access in this execution environment may be restricted for dataset download. If you prefer, I can also produce a pre-filled version with mock/synthetic data so plots are rendered immediately.